In [8]:
import re
import sys
import csv
import pandas
import requests
from lxml import etree
from bs4 import BeautifulSoup
from collections import OrderedDict


class html_parser(object):
    weibo = []
    cookies = {
        'cookie': '_T_WM=94460060722; SCF=Akc6-rgt_orPQPcMl2x5HV85EqkMAXlAtNu4p7Xjq-Yk3pldnC21hCsqvkdyckOmI9VGTWz2-fh41X9Gv2M3pY0.; SUB=_2A25z2CfhDeRhGeBL7FES9i3LzzSIHXVRI0mprDV6PUJbkdANLUb3kW1NRssixUMbX_chzOWyuMU2qvh9yNV6fdq6; SUHB=0Ryq3AMeUc9U3v'}
    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'}

    def __init__(self, filename,  txtname, csvname, wrote_num_txt, wrote_num_csv):
        """
            filename: 爬取到的html页面（保存在txt文件中）
            to_csvname:目标csv路径
            to_txtname:目标txt路径
            wrote_num_txt:txt已写入的记录个数
            wrote_num_csv:csv已写入的记录个数
        """
        self.filename = filename
        self.to_csvname = csvname
        self.to_txtname = txtname
        self.wrote_num_txt = wrote_num_txt
        self.wrote_num_csv = wrote_num_csv

    def get_tree(self):
        """
            生成lxml.etree._Element对象
        """
        try:
            with open(self.filename, mode='r', encoding='utf-8') as f:
                html = f.read()
            # 删除<?xml version="1.0" encoding="UTF-8"?>
            string = '<?xml version="1.0" encoding="UTF-8"?>'
            if string in html:
                html = html.replace(string, '')
            self.html = html
            tree = etree.HTML(html)
            self.tree = tree
        except Exception as e:
            print(e)
            print("get_tree error!")

    def deal_grabled(self, info):
        """
            处理爬取原创微博时的乱码
            info: lxml.etree._Element对象
        """
        try:
            info = (info.xpath('string(.)')).replace(u'\u200b', '').encode(
                sys.stdout.encoding, 'ignore').decode(sys.stdout.encoding)
            return info
        except Exception as e:
            print(e)
            print("deal_grabled error!")

    def is_original(self, info):
        """
            判断一条微博是否为原创微博
            info: lxml.etree._Element对象
        """
        is_original = info.xpath(
            'div/span[@class="cmt"]')  # 原创微博span标签中有class="cmt"属性
        if len(is_original) > 3:
            return False
        else:
            return True

    def get_long_weibo(self, url):
        """
            获取长原创微博
            长原创微博需要再次请求url
            url: 'https://weibo.cn/comment/'+weibo_id
        """
        try:
            r = requests.get(url, headers=self.headers, cookies=self.cookies)
            r.raise_for_status()
            r.encoding = r.apparent_encoding
            html = r.content
            tree = etree.HTML(html)
            info = tree.xpath('//div[@class="c"]')[1]  # 定位
            content = self.deal_grabled(info)  # 处理乱码
            time = info.xpath("//span[@class='ct']/text()")[0]
            weibo_content = content[content.find(':') +
                                    1:content.rfind(time)]  # 分割
            return weibo_content

        except Exception as e:
            print(e)
            print("get_long_weibo error!")

    def get_original_weibo(self, info, weibo_id):
        """
            获取原创微博
            info: lxml.etree._Element对象
            weibo_id: weibo对应的唯一id
        """
        try:
            weibo_content = self.deal_grabled(info)  # 处理乱码
            weibo_content = weibo_content[:weibo_content.rfind(u'赞')]
            a_text = info.xpath('div//a/text()')
            if u'全文' in a_text:  # 长微博判断
                weibo_link = 'https://weibo.cn/comment/' + weibo_id
                content = self.get_long_weibo(weibo_link)
                if content:
                    weibo_content = content
            return weibo_content
        except Exception as e:
            print(e)
            print("get_original_weibo error!")

    def get_repost_weibo(self, info, weibo_id):
        """
            获取转发微博的转发理由
            info: lxml.etree._Element对象
            weibo_id: weibo对应的唯一id
        """
        try:
            reason = self.deal_grabled(info.xpath('div')[-1])
            reason = reason[:reason.rindex(u'赞')]
            weibo_content = reason
            return weibo_content
        except Exception as e:
            print(e)
            print("get_repost_weibo error!")

    def get_weibo_content(self, info, is_original):
        """
            获取微博正文
            info: lxml.etree._Element对象
            is_original: 是否为原创微博 对应原创微博的解析函数
        """
        try:
            weibo_id = info.xpath('@id')[0][2:]
            if is_original:  # 原创微博处理
                weibo_content = self.get_original_weibo(info, weibo_id)
            else:  # 转发微博处理
                weibo_content = self.get_repost_weibo(info, weibo_id)
            print(weibo_content)  # 打印微博正文
            return weibo_content
        except Exception as e:
            print(e)
            print("get_weibo_content error!")

    def get_one_weibo(self, info):
        """
            获取某一条微博的相关信息
        """
        try:
            weibo = OrderedDict()
            is_original = self.is_original(info)  # 是否为原创微博
            weibo['id'] = info.xpath('@id')[0][2:]  # 微博id是第2位以后的字符串
            weibo['content'] = self.get_weibo_content(
                info, is_original)  # 获取微博内容
            weibo['original'] = is_original  # 原创微博标识
            return weibo
        except Exception as e:
            print(e)
            print("get_one_weibo error!")

    def get_all_weibo(self):
        """
            获取一个html页面的全部微博
        """
        try:
            info = self.tree.xpath('//div[@class="c"]')
            is_exist = info[0].xpath('div/span[@class="ctt"]')  # 判断当前页面是否有微博
            if is_exist:
                for i in range(len(info)-2):
                    weibo = self.get_one_weibo(info[i])  # 获取一条微博的信息
                    if weibo:
                        self.weibo.append(weibo)
        except Exception as e:
            print(e)
            print("get_all_weibo error!")

    def write_txt(self):
        """
            将爬取到的微博写入txt文件中
        """
        try:
            result = []
            self.new_txt_num = 0  # 新写入txt记录个数
            for k, i in enumerate(self.weibo):
                result.append('微博id: '+i['id']+'\n'+u'微博正文/转发理由: '+i['content'] +
                              '\n'+'是否原创: '+str(i['original'])+'\n\n')
                self.new_txt_num += 1  # 更新new_txt_num
            self.wrote_num_txt += self.new_txt_num  # 更新wrote_num_txt
            print(result)
            self.result_txt = result
            self.write_result = ''.join(result)
            with open(self.to_txtname, 'a', encoding='utf-8') as f:
                f.write(self.write_result)

            print("write to .txt over!")
            print('write ' + str(self.new_txt_num) +
                  ' weibo ' + 'to .txt file!')
            print('There is already ' +
                  str(self.wrote_num_txt)+' weibo in .txt file!')
        except Exception as e:
            print(e)
            print("write_txt error!")

    def write_csv(self):
        """
            将爬取到的微博写入csv文件中
        """
        self.new_csv_num = 0  # 新写入csv记录个数
        try:
            result_headers = ['微博id', '微博正文/转发理由', '是否原创']
            result_data = [w.values() for w in self.weibo]
            # print(type(result_data))
            # print(len(result_data))
            self.new_csv_num = len(result_data)
            print(result_data)

            with open(self.to_csvname, 'a', encoding='utf-8-sig', newline='') as f:
                writer = csv.writer(f)
                if self.wrote_num_csv == 0:
                    writer.writerows([result_headers])
                writer.writerows(result_data)
            self.wrote_num_csv += self.new_csv_num
            print("write to .csv over!")
            print('write ' + str(self.new_csv_num) +
                  ' weibo ' + 'to .csv file!')
            print('There is already ' +
                  str(self.wrote_num_csv)+' weibo in .csv file!')
        except Exception as e:
            print(e)


In [9]:
test=html_parser('yyqx_txt/yyqx_2.txt','1.txt','2.csv',0,0)

In [10]:
test.get_tree()

In [11]:
test.get_all_weibo()

#奋斗吧青春# 《奋斗的青春最美丽——2020年五·四青年节特别节目》，今晚在CCTV1、CCTV3播出，一起奋斗一起追梦！  [组图共2张] 原图 
转发理由:#五四致敬战疫青年# 致敬战疫青年，致敬最可爱的人！#谢谢你保护了我们#//@TFBOYS组合:#五四致敬战疫青年#五四青年节到来，让我们说一声，#谢谢你保护了我们#  
#朋友请听好# 一起来听小站音乐会 朋友请听好第8期：千玺又被谢娜套路要跳拉丁？ 杨迪沉浸式读信扮演蟑螂笑Skr人  [组图共4张] 原图 
今晚十点，湖南卫视#朋友请听好#，一起来听广播剧。  [组图共2张] 原图 
#华为nova7#系列新品发布会对焦ing @华为终端官方微博 的一直播  [组图共2张] 原图 
问题和烦恼，让书籍给你答案。#423听书节# 来@喜马拉雅  #有声图书馆#，寻找更多未知的答案。  原图 
定义意式优雅，致敬传奇经典。#阿玛尼美妆20周年#，尽在5月14日，#天猫超级品牌日# 阿玛尼的微博视频  
你好，#华为nova7#系列。nova7号电台等你来电 @华为终端官方微博 TFBOYS-易烊千玺的微博视频  
转发理由:和学长在广播小屋相聚了，谢谢何老师  
😆  


In [12]:
test.write_csv()

[odict_values(['J0oMQcPVn', '#奋斗吧青春# 《奋斗的青春最美丽——2020年五·四青年节特别节目》，今晚在CCTV1、CCTV3播出，一起奋斗一起追梦！ \xa0[组图共2张]\xa0原图\xa0', True]), odict_values(['J0keOdHEi', '转发理由:#五四致敬战疫青年# 致敬战疫青年，致敬最可爱的人！#谢谢你保护了我们#//@TFBOYS组合:#五四致敬战疫青年#五四青年节到来，让我们说一声，#谢谢你保护了我们#\xa0\xa0', False]), odict_values(['IFyuniWZ0', '#朋友请听好# 一起来听小站音乐会 朋友请听好第8期：千玺又被谢娜套路要跳拉丁？ 杨迪沉浸式读信扮演蟑螂笑Skr人 \xa0[组图共4张]\xa0原图\xa0', True]), odict_values(['IEH5zxf1L', '今晚十点，湖南卫视#朋友请听好#，一起来听广播剧。 \xa0[组图共2张]\xa0原图\xa0', True]), odict_values(['IEGAgq2eU', '#华为nova7#系列新品发布会对焦ing @华为终端官方微博 的一直播 \xa0[组图共2张]\xa0原图\xa0', True]), odict_values(['IEyyFiGxr', '问题和烦恼，让书籍给你答案。#423听书节# 来@喜马拉雅  #有声图书馆#，寻找更多未知的答案。 \xa0原图\xa0', True]), odict_values(['IDIgZ3EB7', '定义意式优雅，致敬传奇经典。#阿玛尼美妆20周年#，尽在5月14日，#天猫超级品牌日# 阿玛尼的微博视频 \xa0', True]), odict_values(['IDAW6nFrE', '你好，#华为nova7#系列。nova7号电台等你来电 @华为终端官方微博 TFBOYS-易烊千玺的微博视频 \xa0', True]), odict_values(['IDztOzhIe', '转发理由:和学长在广播小屋相聚了，谢谢何老师\xa0\xa0', False]), odict_values(['IDtcMcicK', '😆 \xa0', True])]
write to .csv over!
write 10 we